In [9]:
import csv
import time

# Load data: user_id -> set of item_ids
def load_data(file_path):
    user_item_data = {}

    with open(file_path, newline="", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row

        for user_id, item_id in reader:
            if user_id not in user_item_data:
                user_item_data[user_id] = set()
            user_item_data[user_id].add(item_id)

    return user_item_data

In [10]:
class HashTable:
    def __init__(self, size, collision_avoidance):
        # For separate chaining: each slot is a list (bucket)
        # For probing: each slot is either None or (key, [items])
        if collision_avoidance == "separate_chaining":
            self.table = [[] for _ in range(size)]
        else:
            self.table = [None] * size

        self.size = size
        self.collisions = 0
        self.collision_avoidance = collision_avoidance
        self.count = 0  # number of keys stored

    def hash(self, key):
        return hash(key) % self.size

    # Provided for completeness; not used for separate_chaining/linear_probing
    def second_hash(self, key):
        return 1 + (hash(key) % (self.size - 1))

    def insert(self, key, value):
        """
        Insert a (user, item) pair.
        If user already exists, add item to their list.
        """
        index = self.hash(key)

        if self.collision_avoidance == "separate_chaining":
            bucket = self.table[index]

            # Look for existing key
            for i, (k, items) in enumerate(bucket):
                if k == key:
                    if value not in items:
                        items.append(value)
                        bucket[i] = (k, items)
                    return

            # No key found — insert new
            if len(bucket) > 0:
                self.collisions += 1  # bucket already had something

            bucket.append((key, [value]))
            self.count += 1

        elif self.collision_avoidance == "linear_probing":
            original_index = index

            while True:
                entry = self.table[index]

                if entry is None:
                    # Insert new key here
                    self.table[index] = (key, [value])
                    self.count += 1
                    return

                existing_key, items = entry

                if existing_key == key:
                    # Same key — add item if not already present
                    if value not in items:
                        items.append(value)
                        self.table[index] = (existing_key, items)
                    return

                # Collision with a different key
                self.collisions += 1
                index = (index + 1) % self.size

                if index == original_index:
                    raise RuntimeError("HashTable is full during insert.")

        else:
            raise ValueError("Unknown collision avoidance technique.")

    def retrieve(self, key):
        """
        Return list of items stored for this user.
        """
        index = self.hash(key)

        if self.collision_avoidance == "separate_chaining":
            bucket = self.table[index]
            for (k, items) in bucket:
                if k == key:
                    return items
            return []

        elif self.collision_avoidance == "linear_probing":
            original_index = index

            while True:
                entry = self.table[index]
                if entry is None:
                    return []

                existing_key, items = entry
                if existing_key == key:
                    return items

                index = (index + 1) % self.size
                if index == original_index:
                    return []

        else:
            raise ValueError("Unknown collision avoidance technique.")


In [11]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    def parent(self, index):
        return (index - 1) // 2

    def left_child(self, index):
        return 2 * index + 1

    def right_child(self, index):
        return 2 * index + 2

    def has_left(self, index):
        return self.left_child(index) < len(self.heap)

    def has_right(self, index):
        return self.right_child(index) < len(self.heap)

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def percolate_up(self, index):
        parent = self.parent(index)
        if index > 0 and self.heap[index][0] > self.heap[parent][0]:
            self.swap(index, parent)
            self.percolate_up(parent)

    def percolate_down(self, index):
        largest = index
        left = self.left_child(index)
        right = self.right_child(index)

        if self.has_left(index) and self.heap[left][0] > self.heap[largest][0]:
            largest = left
        if self.has_right(index) and self.heap[right][0] > self.heap[largest][0]:
            largest = right
        if largest != index:
            self.swap(index, largest)
            self.percolate_down(largest)

    def push(self, priority, item):
        self.heap.append((priority, item))
        self.percolate_up(len(self.heap) - 1)

    def pop(self):
        if len(self.heap) == 0:
            return None
        if len(self.heap) == 1:
            return self.heap.pop()

        self.swap(0, len(self.heap) - 1)
        item = self.heap.pop()
        self.percolate_down(0)
        return item

    def top_n(self, n):
        top_items = []
        count = min(n, len(self.heap))
        for _ in range(count):
            priority, item = self.pop()
            top_items.append(item)
        return top_items

In [12]:
class RecommendationSystem:
    def __init__(self, user_item_file):
        # Load user -> set of items from the CSV file
        self.user_item_file = user_item_file
        self.user_item_data = load_data(user_item_file)

    def build_recommendation_system(self, technique, size):
        """
        Build the hash table for the given collision technique.
        Returns the hash table and insertion time.
        """
        hash_table = HashTable(size, technique)

        start_time = time.time()

        # Insert all (user, item) pairs
        for user, items in self.user_item_data.items():
            for item in items:
                hash_table.insert(user, item)

        end_time = time.time()
        insertion_time = end_time - start_time

        return hash_table, insertion_time

    def recommend_items_with_jaccard(self, target_user, technique, top_n):
        """
        Make recommendations for target_user using:
        - HashTable (technique)
        - Jaccard similarity
        - MaxHeap

        Returns: (recommendations, insertion_time, retrieval_time, collisions)
        """
        table_size = 20  # small data, simple size

        # 1. Build hash table
        hash_table, insertion_time = self.build_recommendation_system(
            technique, table_size
        )

        # 2. Compute similarities and item scores
        start_time = time.time()

        target_items = set(hash_table.retrieve(target_user))

        item_scores = {}  # item -> accumulated similarity

        for user, items_from_dict in self.user_item_data.items():
            if user == target_user:
                continue

            other_items = set(hash_table.retrieve(user))

            # Jaccard similarity
            intersection = target_items.intersection(other_items)
            union = target_items.union(other_items)

            if len(union) == 0:
                similarity = 0.0
            else:
                similarity = len(intersection) / len(union)

            if similarity == 0:
                continue

            # Accumulate scores for items the target user does NOT already own
            for item in other_items:
                if item in target_items:
                    # DO NOT recommend items they already own
                    continue

                if item not in item_scores:
                    item_scores[item] = 0.0
                item_scores[item] += similarity

        # 3. Push items into MaxHeap
        heap = MaxHeap()
        for item, score in item_scores.items():
            heap.push(score, item)

        recommendations = heap.top_n(top_n)

        end_time = time.time()
        retrieval_time = end_time - start_time

        collisions = hash_table.collisions

        return recommendations, insertion_time, retrieval_time, collisions

In [13]:
def main():
    user_item_file = "user_item_data.csv"

    # Create the recommendation system
    rec_sys = RecommendationSystem(user_item_file)

    # Collision techniques we are comparing
    techniques = ["separate_chaining", "linear_probing"]

    # Get list of users from the data
    target_users = sorted(rec_sys.user_item_data.keys())

    for target_user in target_users:
        print(f"\nRunning recommendations for target user: {target_user}\n")

        for technique in techniques:
            recommendations, insertion_time, retrieval_time, collisions = \
                rec_sys.recommend_items_with_jaccard(
                    target_user=target_user,
                    technique=technique,
                    top_n=3  # top 3 items
                )

            print(f"Using collision avoidance technique: {technique}")
            print(f"Recommendations for {target_user}: {recommendations}")
            print(f"Insertion Time: {insertion_time:.6f} seconds")
            print(f"Retrieval Time: {retrieval_time:.6f} seconds")
            print(f"Collisions: {collisions}\n")


if __name__ == "__main__":
    main()



Running recommendations for target user: user1

Using collision avoidance technique: separate_chaining
Recommendations for user1: ['The Last of Us', 'Battlefield', 'Gears of War']
Insertion Time: 0.000114 seconds
Retrieval Time: 0.000219 seconds
Collisions: 1

Using collision avoidance technique: linear_probing
Recommendations for user1: ['The Last of Us', 'Battlefield', 'Gears of War']
Insertion Time: 0.000071 seconds
Retrieval Time: 0.000092 seconds
Collisions: 37


Running recommendations for target user: user2

Using collision avoidance technique: separate_chaining
Recommendations for user2: ['Bayonetta', 'Splatoon', 'Gears of War']
Insertion Time: 0.000081 seconds
Retrieval Time: 0.000075 seconds
Collisions: 1

Using collision avoidance technique: linear_probing
Recommendations for user2: ['Bayonetta', 'Splatoon', 'Gears of War']
Insertion Time: 0.000062 seconds
Retrieval Time: 0.000071 seconds
Collisions: 37


Running recommendations for target user: user3

Using collision avoid

## Collision Technique Analysis

In my experiments I compared two collision resolution techniques:

- **Separate Chaining**
- **Linear Probing**

### Which had fewer collisions?

From the output, **separate chaining** had the fewest collisions (around 1), while **linear probing** had many more (around 30–40).  

This makes sense because:

- In **separate chaining**, each table slot holds a *bucket* (a small list).  
  When a collision happens, the new key just gets added to that list.  
  We still count it as a collision, but we do **not** have to move to another index in the table.
- In **linear probing**, when a collision happens we keep stepping forward through the array until we find an empty spot.  
  This creates *primary clustering*, where groups of occupied cells grow and cause even more collisions as the table fills.

So linear probing naturally reports more collisions because it has to search across multiple slots whenever several keys hash to nearby positions.

### Strengths and weaknesses

**Separate Chaining**

- **Strengths**
  - Simple to implement.
  - Handles high load factors better (the table can be quite full and still work, because extra items just go in the bucket list).
  - Fewer clustering issues.
- **Weaknesses**
  - Uses extra memory for the lists in each bucket.
  - Worst-case lookup time can be slower if one bucket’s list gets long.

**Linear Probing**

- **Strengths**
  - Everything is stored directly in the array, so it is memory-friendly and cache-friendly.
  - Easy to implement and doesn’t require extra data structures.
- **Weaknesses**
  - More sensitive to load factor: as the table gets full, performance drops quickly.
  - Suffers from clustering, which increases the number of probes and collisions.

In my results, this shows up as **many more collisions** for linear probing compared to separate chaining.

### Was MaxHeap the best choice?

Using a **MaxHeap** for recommendations is a reasonable choice because:

- We assign each candidate item a **score** (sum of similarity values).
- We want to efficiently get the **top N items** with the highest scores.
- A max-heap supports:
  - `push` in \(O(\log n)\) time
  - `pop` of the max item in \(O(\log n)\) time
  - Getting the top \(N\) recommendations in \(O(N \log n)\) time

Another option would be:

- Put all `(score, item)` pairs into a regular **list** and call `sorted(...)` or `list.sort(...)`.  
  This is \(O(n \log n)\) and is perfectly fine for small data, but for larger data a heap can be more efficient if we only need the top few elements.
- Use a **balanced tree** or **ordered structure**, but that would be more complex to implement by hand.

For this project, the **MaxHeap is a good choice** because it matches the idea of “priority recommendations” and lets us practice implementing a heap data structure ourselves. For small datasets, sorting would also work, but the heap gives us better practice with priority queues and aligns with the assignment’s goals.
